# Notes

https://pandas.pydata.org/docs/reference/api/pandas.testing.assert_series_equal.html

https://realpython.com/python-assert-statement/#:~:text=in%20your%20code.-,What%20Are%20Assertions%3F,while%20you're%20debugging%20code.

#### Useful functions

- Include also non-numeric values

`df.describe(include="all")`
- Count number of distinct elements

0=rows [Default], 1=headers
`df.nunique(axis=0)`

- plot barchart with group by

*By default, pandas is using `matplotlib`*
`df.groupby(by=["day"]).sum()[["tip"]].sort_values(by=["tip"]).plot(kind="bar")`

*Use `plotly` for the plotting backend*
`pd.options.plotting.backend = "plotly"
df.groupby(by=["day"]).sum()[["tip"]].sort_values(by=["tip"]).plot(kind="bar")`

- Pandas & Numpy Where function

`df["tip"].where(df["tip"] > 2)`

*replacing NAN in above with mean*

``avg = df["tip"].mean()
df_copy = df.copy()
df_copy["tip"] = df["tip"].where(df["tip"] >= avg, "below average")
df_copy.head()``

- You can also use `inplace=True` to apply the changes to the dataframe directly

 ```df['tip'].where(df['tip'] >= avg, "below average", inplace=True)```
 
 
 - Logical selection (filter)
 
 `df["is_weekend"] = np.where(df["day"].isin(["Sat", "Sun"]), True, False)
df.head()`

Style your dataframe

In [ ]:
# Find more options here: https://pandas.pydata.org/docs/user_guide/style.html
def make_pretty(styler):
    styler.set_caption("Tips in USD")
    styler.format({"total_bill": "USD {:.1f}", "tip": "USD {:.1f}", "size": "{} seats"})
    styler.background_gradient(cmap="RdYlGn", subset=["total_bill", "tip"])
    styler.set_table_styles([{"selector": "th.col_heading", "props": [("background-color", "#d3d3d3")]}])
    styler.hide(subset=["sex", "smoker"], axis=1)
    styler.bar(subset=["size"])

    return styler

In [ ]:
df.head().style.pipe(make_pretty)
# When exporting to Excel not all styles will be kept
df.head().style.pipe(make_pretty).to_excel("styled_output.xlsx", engine="openpyxl", index=False)

https://github.com/Sven-Bo/five-little-known-pandas-features/blob/master/Little_Known_Pandas_Features.ipynb

# Data Science Starter (Ken Jee)

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 15 12:52:51 2020
@author: Ken
"""

# import exploration files 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

file_path = 'path_to_data.csv'

# read in data 
data = pd.read_csv(file_path)


############################################################################## 
#Data Exploration
##############################################################################

#rows and columns returns (rows, columns)
data.shape

#returns the first x number of rows when head(num). Without a number it returns 5
data.head()

#returns the last x number of rows when tail(num). Without a number it returns 5
data.tail()

#returns an object with all of the column headers 
data.columns

#basic information on all columns 
data.info()

#gives basic statistics on numeric columns
data.describe()

#shows what type the data was read in as (float, int, string, bool, etc.)
data.dtypes

#shows which values are null
data.isnull()

#shows which columns have null values
data.isnull().any()

#shows for each column the percentage of null values 
data.isnull().sum() / data.shape[0]

#plot histograms for all numeric columns 
data.hist() 


############################################################################## 
#Data Manipulation
##############################################################################

# rename columns 
data.rename(index=str columns={'col_oldname':'col_newname'})

# view all rows for one column
data.col_name 
data['col_name']

# multiple columns by name
data[['col1','col2']]
data.loc[:['col1','col2']]

#columns by index 
data.iloc[:,[0:2]]

# drop columns 
data.drop('colname', axis =1) #add inplace = True to do save over current dataframe
#drop multiple 
data.drop(['col1','col2'], axis =1)

#lambda function 
data.apply(lambda x: x.colname**2, axis =1)

# pivot table 
pd.pivot_table(data, index = 'col_name', values = 'col2', columns = 'col3')

# merge  == JOIN in SQL
pd.merge(data1, data2, how = 'inner' , on = 'col1')

# write to csv 
data.to_csv('data_out.csv')

### Ken Jee Fundamentals

data cleaning

In [ ]:
# data cleaning

# -*- coding: utf-8 -*-
"""
Created on Mon Jul 15 08:27:42 2019
@author: KJee
"""

import pandas as pd 

data = pd.read_csv('craigslistVehicles.csv')
data.columns

data.describe()

#remove duplcates 
data.drop_duplicates(inplace= True)

#check for nulls / % of nulls 

data.isnull().any()
data.isnull().sum()/ data.shape[0]

#remove columns with certain threshold of nulls
#threshold is the number of columns or rows without nulls 
thresh = len(data)*.6
data.dropna(thresh = thresh, axis = 1)
data.dropna(thresh = 21, axis = 0)

#imputing nulls fillna()
data.odometer.fillna(data.odometer.median())
data.odometer.fillna(data.odometer.mean())

#everything lower or uppercase
data.desc.head()
data.desc.head().apply(lambda x: x.lower())
data.desc.head().apply(lambda x: x.upper())

#use regex .extract
#use strip()
#use replace()
#split 

data.cylinders.dtype
data.cylinders.value_counts()
data.cylinders = data.cylinders.apply(lambda x: str(x).replace('cylinders','').strip())
data.cylinders.value_counts()

#change data type 
data.cylinders = pd.to_numeric(data.cylinders, errors = 'coerce')


#boxplot 
data.boxplot('price')
data.boxplot('odometer')

#outlier detection and normalization remove rows with > 99% / z score 
numeric = data._get_numeric_data()

# with no null values 
from scipy import stats
import numpy as np 

data_outliers = data[(data.price < data.price.quantile(.995)) & (data.price > data.price.quantile(.005))]

data_outliers.boxplot('price')

#remove duplcates, subset, keep, etc.
data.drop_duplicates()

#histogram
data_outliers.price.hist()

#types of normalization 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data.cylinders.values.reshape(-1,1))
scaler.transform(data.cylinders.values.reshape(-1,1))

data exploration

In [ ]:
# data exploration
# -*- coding: utf-8 -*-
"""
Created on Tue Jul  2 21:46:40 2019
@author: Ken
"""



""" What is Pandas?
Pandas is a very flexible data science library that allows us to load in data, manipulate it, and analyze it. 
- Pandas uses dataframes which we can think of as tables. We can use functions to create or manipulate rows and columns. 
- Using pandas dataframes, we can begin to understand trends in data we can perform functions on the rows or columns to understand more about our set
- Pandas also has some visualization tools that we can use to better understand our data
"""

import pandas as pd 

# read in data from workign directory (folder in top right)
# can read in from anywhere if full path is the pd.read_csv()
data = pd.read_csv('craigslistVehicles.csv')

#rows and columns returns (rows, columns)
data.shape

#returns the first x number of rows when head(num). Without a number it returns 5
data.head()

#returns the last x number of rows when tail(num). Without a number it returns 5
data.tail()

#returns an object with all of the column headers 
data.columns

#basic information on all columns 
data.info()

#gives basic statistics on numeric columns
data.describe()

#shows what type the data was read in as (float, int, string, bool, etc.)
data.dtypes

#shows which values are null
data.isnull()

#shows which columns have null values
data.isnull().any()

#shows for each column the percentage of null values 
data.isnull().sum() / data.shape[0]

# for categorical variables 

#shows unique values that appear in the column 
#data.type = data['type']
data.type.unique()

#shows the counts for those unique values 
data.type.value_counts()

#shows the percentage of values from 
data.type.value_counts()/ data.type.notnull().sum()

data.cylinders.head(10)

############################################################# Graphing #######################################################
#histogram of year 
# data.year.hist() == data.year.plot(kind='hist')
data.year.hist()
data.year.hist(bins=100)

#bar chart of types 
data.type.value_counts().plot(kind='bar')

data manipulation

In [ ]:
# data manipulation

# -*- coding: utf-8 -*-
"""
Created on Mon Jul  8 10:17:05 2019
@author: KJee
"""

#load in the pandas module 
import pandas as pd 

# read in data from workign directory (folder in top right)
# can read in from anywhere if full path is the pd.read_csv()
data = pd.read_csv('craigslistVehicles.csv')

#view columns & rename columns 
data.columns
data.rename(index=str,columns={"url":"new_url"})
data.rename(index=str,columns={"new_url":"url"})

# view all rows for one column
data['url']

#view all columns for select group of rows
data[0:10]

#filter for multiple columns (all below do the same thing ) 
data[['url','city','price']]
data.loc[:,['url','city','price']]
data.iloc[:,0:3]

#filter by rows and columns 
data.loc[0:100,['url','city','price']]
data.iloc[0:100,0:3]

#filter by column list 
data[data.columns]


#drop / add column #inplace = True 
#axis & inplace 
data.drop('url', axis = 1)
data.drop(['url','price'], axis = 1)

#add column 
data['age'] = 2019 - data['year']

#filtering data by columns & boolean indexing 
data[(data['age'] < 5)]

data.loc[(data.age <5),:]

# basic operators on columns 
data['price_per_mile'] = data['price'] / data['odometer']

# apply function 

def timex2(x):
    return 2*x

data['price2x'] = data['price'].apply(timex2)
data['price'].head()
data['price2x'].head()

#lambda function 
data['price3x'] = data['price'].apply(lambda x: x*3)
data['price3x'].head()

#tenary operator 
data['expensive'] = data['price'].apply(lambda x: 'expensive' if x > 10000 else 'cheap')

data['newandcheap'] = data.apply(lambda x: 'yes' if x['price'] < 10000 and x['age'] < 5 else 'no', axis = 1)
data['newandcheap2'] = data[['price','age']].apply(lambda x: 'yes' if x[0] < 10000 and x[1] < 5 else 'no', axis = 1)

#binning pd.cut / pd.qcut
pd.qcut(data.price,5) #even number 
pd.cut(data.price,5 ) #even spacing 

#dummy variables 
data_dummies = pd.get_dummies(data[['price','year','fuel','transmission','type']])

#pivot table / sort_index / sort_values 
data.pivot_table(index='year',columns='type',values='price',aggfunc ='mean').sort_index(ascending=False)
data.pivot_table(index='year',columns='type',values='price',aggfunc ='count').sort_index(ascending=False)
data.pivot_table(index='year',columns='type',values='price',aggfunc ='count').sort_index(ascending=False).plot()

#groupby 
data.groupby('type').mean()
data.groupby(['type','fuel']).mean()
data.groupby(['type','fuel'],as_index = False).mean()

# pd.merge == to join in sql  
df1 = data[['url','city']]
df2 = data[['url','price']]

df_merged = pd.merge(df1,df2,on='url')


#append and concatenate (pd.concat / pd.append)
data100 = data.sample(100, random_state = 1)
data1002 = data.sample(100, random_state = 2)

data100.append(data1002)
pd.concat([data100,data1002], axis = 0)

# write to a csv file pd.to_csv()
data100.to_csv('data100.csv')

regression

In [ ]:
# regression
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 11 12:37:26 2019
@author: Ken
"""

# -*- coding: utf-8 -*-
"""
Created on Mon Jul 15 08:27:42 2019
@author: KJee
"""

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

data = pd.read_csv('craigslistVehicles.csv')
data.columns

data.describe()

#remove duplcates 
data.drop_duplicates(inplace= True)

#check for nulls / % of nulls 

data.isnull().any()
data.isnull().sum()/ data.shape[0]

#remove columns with certain threshold of nulls
#threshold is the number of columns or rows without nulls 
"""thresh = len(data)*.6
data.dropna(thresh = thresh, axis = 1)
data.dropna(thresh = 21, axis = 0)
"""

#everything lower or uppercase
data.desc.head()
data.desc.head().apply(lambda x: x.lower())
data.desc.head().apply(lambda x: x.upper())
data['text_len'] = data.desc.apply(lambda x: len(str(x)))
(data['text_len'].value_counts() > 1).sum()

data.cylinders.dtype
data.cylinders.value_counts()
data.cylinders = data.cylinders.apply(lambda x: str(x).replace('cylinders','').strip())
data.cylinders.value_counts()

#change data type 
data.cylinders = pd.to_numeric(data.cylinders, errors = 'coerce')
data.cylinders.value_counts()

#boxplot 
data.boxplot('price')
data.boxplot('odometer')

data.price.max()
data.odometer.max()

#outlier detection and normalization remove rows with > 99% / z score 

# with no null values 
data_outliers = data[(data.price < data.price.quantile(.995)) & (data.price > data.price.quantile(.005)) & (data.price != 0) & (data.odometer != 0)]
data_outliers = data_outliers[(data_outliers.odometer < data_outliers.odometer.quantile(.995)) & (data_outliers.odometer > data_outliers.odometer.quantile(.005))]

#histogram
data_outliers[['price','odometer','cylinders','text_len']].hist()

#types of data cleaning  
data_outliers.isnull().sum()/data_outliers.shape[0]

#imputing nulls fillna()

data_outliers.dropna(subset=['manufacturer','make','fuel','transmission', 'title_status','year'], inplace = True)
data_outliers.isnull().sum()/data_outliers.shape[0]

data_outliers.cylinders.fillna(data_outliers.cylinders.median(), inplace = True)
data_outliers.isnull().sum()/data_outliers.shape[0]

data_outliers[['condition','VIN','drive','type','paint_color']]= data_outliers[['condition','VIN','drive','type','paint_color']].fillna('n/a')
data_outliers.isnull().sum()/data_outliers.shape[0]

data_outliers.VIN = data_outliers.VIN.apply(lambda x: 'has_vin' if x != 'n/a' else 'no_vin' )

data_final = data_outliers.drop(['city_url','url','city','size','desc','lat','long','image_url'], axis = 1)
data_final['constant'] = 1
data_final['age'] = 2019 - data_final.year
data_final.isnull().any()

numeric = data_final._get_numeric_data()

import seaborn as sns

corrdata = numeric

corr = corrdata.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

#simple linear regression for year
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

#set variables need to be in specific format 
X1 = data_final.odometer.values.reshape(-1,1)
y1 = data_final.price.values.reshape(-1,1)

#create train / test split for validation 
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, random_state=0)
        
reg = LinearRegression().fit(X_train1, y_train1)
reg.score(X_train1, y_train1)
reg.coef_
y_hat1 = reg.predict(X_train1)

plt.scatter(X_train1,y_train1)
plt.scatter(X_train1,y_hat1)
plt.show()

y_hat_test1 = reg.predict(X_test1)
plt.scatter(X_test1, y_test1)
plt.scatter(X_test1, y_hat_test1)
plt.show()

#MSE & RMSE penalize large errors more than MAE 
mae = mean_absolute_error(y_hat_test1,y_test1)
rmse = math.sqrt(mean_squared_error(y_hat_test1,y_test1))
print('Root Mean Squared Error = ',rmse)
print('Mean Absolute Error = ',mae)

import statsmodels.api as sm

X1b = data_final[['constant','odometer']]
y1b = data_final.price.values

X_train1b, X_test1b, y_train1b, y_test1b = train_test_split(X1b, y1b, test_size=0.3, random_state=0)

reg_sm1b = sm.OLS(y_train1b, X_train1b).fit()
reg_sm1b.summary()


#multiple linear regression 
from statsmodels.stats.outliers_influence import variance_inflation_factor

X2 = data_final[['constant','age','odometer','cylinders','text_len']]
y2 = data_final.price.values

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=0)

reg_sm2 = sm.OLS(y_train2, X_train2).fit()
reg_sm2.summary()

pd.Series([variance_inflation_factor(X2.values,i) for i in range(X2.shape[1])],index=X2.columns)



#actual regression 
X3 = pd.get_dummies(data_final[['constant','age','odometer','text_len','cylinders','condition','fuel','VIN','type']])
y3 = data_final.price.values

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.3, random_state=0)

reg_sm3 = sm.OLS(y_train3, X_train3).fit()
reg_sm3.summary()

y_hat3 = reg_sm.predict(X_test3)

rmse3 = math.sqrt(mean_squared_error(y_hat3,y_test3))

plt.scatter(y_hat3,y_test3)

#cross validation 5 fold 
from sklearn.model_selection import cross_val_score 
X4 = pd.get_dummies(data_final[['age','odometer','cylinders','condition','fuel','VIN','type']])
y4 = data_final.price.values

X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size=0.3, random_state=0)

reg4 = LinearRegression().fit(X_train4, y_train4)
reg4.score(X_train4,y_train4)

scores = cross_val_score(reg4,X4,y4, cv=5, scoring = 'neg_mean_squared_error')
np.sqrt(np.abs(scores))



# Data Prof
https://github.com/dataprofessor

## README
An ideal README should contain the following sections:

- A proper description of what the project is about and what it does
- List of software dependencies required to run the project and how to install them
- Step by Step details about how to run the project



## Udacity Git Commit Message Style Guide
### Commit Messages

**Message Structure**

A commit message consists of three distinct parts separated by a blank line: the title, an optional body and an optional footer. The layout looks like this:

type: subject

body

footer
The title consists of the type of the message and subject.

**The Type**
The type is contained within the title and can be one of these types:

- feat: a new feature
- fix: a bug fix
- docs: changes to documentation
- style: formatting, missing semi colons, etc; no code change
- refactor: refactoring production code
- test: adding tests, refactoring test; no production code change
- chore: updating build tasks, package manager configs, etc; no production code change

**The Subject**
Subjects should be no greater than 50 characters, should begin with a capital letter and do not end with a period.

Use an imperative tone to describe what a commit does, rather than what it did. For example, use change; not changed or changes.

**The Body**
Not all commits are complex enough to warrant a body, therefore it is optional and only used when a commit requires a bit of explanation and context. Use the body to explain the what and why of a commit, not the how.

When writing a body, the blank line between the title and the body is required and you should limit the length of each line to no more than 72 characters.

**The Footer**
The footer is optional and is used to reference issue tracker IDs.

**Example Commit Message**
feat: Summarize changes in around 50 characters or less

More detailed explanatory text, if necessary. Wrap it to about 72 characters or so. In some contexts, the first line is treated as the subject of the commit and the rest of the text as the body. The blank line separating the summary from the body is critical (unless you omit the body entirely); various tools like log, shortlog and rebase can get confused if you run the two together.

Explain the problem that this commit is solving. Focus on why you are making this change as opposed to how (the code explains that). Are there side effects or other unintuitive consequences of this change? Here's the place to explain them.

Further paragraphs come after blank lines.

Bullet points are okay, too
Typically a hyphen - or asterisk * is used for the bullet, preceded by a single space, with blank lines in between, but conventions vary here
If you use an issue tracker, put references to them at the bottom, like this:
Resolves: #123
See also: #456, #789

## for reference to github guide Udacity
[See](https://docs.google.com/document/d/e/2PACX-1vSkShiwfp8DP79yO1wiGI2cyDqlzMugMdivXzMfr_6CVESvbLAVYeiZowV47VJMRQ_L4F4fdeQjo9qd/pub)


### How to write a good README File?

https://www.mygreatlearning.com/blog/readme-file/

As we have understood that a good readme file is necessary, let’s look at writing a good readme file. When writing a readme file, the main focus is it should be brief but also contain all the information needed. In addition, it should have clarity and need to be structured. Some points to help you in writing:

**Up-to-date**

A readme file should be regularly updated to not conflict with what is uploaded to the repository. If possible, a readme file should contain details about all the version uploads and mention the changes/ updates to the project.

**Brief and Clear**: A readme file should not extend like an essay. It should only have what is necessary and not anything more. It should be written clearly, keeping in mind the reader who is going to read. It should be written in a simple language so that everyone can understand. It is better to use bullet points or tables to convey your points better. 

**Detailed**: The readme file should not miss out on any information. It should contain all the necessary details necessary to understand what the project is about and how to use it. For example, if there exists a need to install software to run the project. Then the version of the software and technical configurations needed to run the project on their computer should be mentioned in full. If you feel the readme file is getting too long, you can break it up and just show what is in the context of the page or folder of the repository. You can also provide links to tutorials or instructions to configure the system or install the necessary software to make it short.

**Self Explanatory**:  The user shouldn’t need to refer to some other article to understand your readme file. It should be self contained, as in the user should be able to understand everything by reading it. So it’s best to write everything in layman terms for the common reader to understand because you never know who is going to read it. As developers, we do try to avoid writing the basics because it’s quite normal. But it’s best to write it as the reader might be a beginner in the field.
What is a good template to write a README File?
There are multiple layouts or formats on the web which will help you in writing a readme file. No matter which template you follow, make sure that it follows all the points listed above. So let’s just look at one of them:

**Project Title**
A little brief about what the project is about. It should be like a small summary format informing about the main purpose of the project.

Motivation

This section is for letting the reader know why you created this project, the reason behind pursuing such a project, and why you have decided to do it.

Build Status

This basically explains the current build status of the project. If there is a bug /error which needs addressing. This is done so for two different reasons The user understands that this is an issue and does not spend more time figuring if it was a mistake on their part.A developer who is familiar with the issue can suggest some solutions directly without going through the whole code.

Code Style

This lets the users know that you have used a particular code style and helps them when contributing to your project so that the whole project code style stays the same. Some common code styles: standard, xo, etc.

Screenshots

As the saying goes, a picture is equal to a thousand words. Most people will be interested if there is a visual representation of what the project is about. It helps them understand better. A visual representation can be snapshots of the project or a video of the functioning of the project.

Tech/Framework used

This is used to help the reader understand which tech or frameworks have been used to do the project. It helps the reader understand which all tech stack he has to be familiar with to understand the whole project.

Features

This is where you write what all extra features have been done in your project. Basically this is where you try to make your project stand out from the rest.

Code Examples

This is where you try to compress your project and make the reader understand what it does as simply as possible. This should help the reader understand if your code solves their issue.

Installation

If your project needs installation of certain software or configurations to the system. Do mention it in this section as it helps a lot for the reader to use your project. The steps mentioned should be precise and explanatory.  If possible, you can add links that can help them better understand how to configure/install the necessary files or softwares.

API reference

If your project is small, then we can add the reference docs in the readme. For larger projects, it is better to provide links to where the API reference documentation is documented.

Tests

This is the section where you mention all the different tests that can be performed with code examples

How to Use?

As I have mentioned before, you never know who is going to read your readme. So it is better to provide information on how to use your project. A step-by-step guide is best suited for this purpose. It is better to explain steps as detailed as possible because it might be a beginner who is reading it.

Contribute

This is where you let them know that they can contribute and help you out. A guideline on how to contribute is also helpful

Credits

Giving proper credit is most important. Mention any links/repos which helped you or inspired you to build this project. It can be a blog, another open source project, or just people who have contributed in building this project.

License

A short description of the license. (MIT, Apache, etc.)

How to make your README file more interesting?

Now that you have understood what to write in a readme. Let’s look at how to make it interesting to read. The main issue with readme files is it can look like a terms and conditions page of a product. It is a long page filled with information. This is not at all interesting to the user. So some methods to improve your readme file is by adding images and visual representations. Having images helps it become more colorful and more attractive. It also helps in better understanding for some people. Everyone might not be familiar with all the technical jargon which we use. So showing visual representation helps more in that case. Adding funny-related memes or videos also helps in some ways.

How is README written?
We have all seen what all to write. But we did not discuss how to write it. In the beginning, we mentioned that the readme file is in md format, which stands for markdown documentation. So let’s see how we can write it.

Markdown is very similar to text with some syntax written for bold, italic, links etc.

So let’s head on with some examples:

Headers:
#   This is a  <h1> tag.  ##  This is a  <h2> tag. ###### This is a <h6> tag.
Result: This is a <h1> tag.This is a  <h2> tag.This is a <h6> tag.
Emphasis:
*This text will be italic*_This will also be italic_
**This text will be bold**__This will also be bold__
**You **can** combine them*
Result:This text will be italicThis will also be italic
This text will be boldThis will also be bold
You can combine them 
Block Quotes:    
As Grace Hopper said: > I’ve always been more interested > in the future than in the present.Result:
Lists:
Unordered Lists:
Item 1Item 2Item 3Item 4

Order Lists:
Item 1Item 2Item 3Item 4Item 5
Images:
![GitHub Logo](https://d1m75rqqgidzqn.cloudfront.net/images/logo.png) Format: ![Alt Text](url) 
Links:
http://github.com – automatic! [GitHub](http://github.com) 
Extras 
\ –  backslash
‘ – backtick
* – asterisk
_ – underscore
{} – curly braces
[] – square brackets
() – parenthesis
# – hash mark
+ – plus sign
– – minus sign
. – dot
! – exclamation mark
Code
Github also supports something called code fencing, which allows for multiple lines without indentation:“`if (isAwesome){  return true}Result:
Task Lists
But I have to admit, tasks lists are my favourite:
– [x] This is a complete item– [ ] This is an incomplete item
Result:
Tables
First Header | Second Header———— | ————-Content from cell 1 | Content from cell 2Content in the first column | Content in the second column
Result:
SHA References
If we enter a commit SHA-1 hash, it will be automatically converted to a link by Github.

Issues references within a repository
Any reference to any issue or pull request is automatically converted to a link.

UserName mentions
If there is a need to mention someone who contributed in any format to the project or reported an issue, then you can easily mention that person using the ‘@’ symbol followed by the name. You can also refer to teams by using the @ keyword.

Automatic URL conversion
Any URL link which is entered is converted directly into a clickable link.

Strikethrough
Any text or word wrapped in ~~word~~ will appear as crossed out.

Emoji
Now we can enter emojis as well to express better.

Some common emojis are as follows:

SNo.	ShortCode	Emoji		ShortCode	Emoji
1	:grinning:	😀	8	:smiley:	😃
2	🙂	😄	9	😀	😁
3	:laughing: or :satisfied:	😆	10	:sweat_smile:	😅
4	:rofl:	🤣	11	:joy:	😂
5	:slightly_smiling_faces:	🙂	12	:upside_down_face:	🙃
6	😉	😉	13	:blush:	😊
7	:innocent:	😇			
Now that we have seen how to write in the md file . Let’s look at how the readme file is intended for different users.

For end-users: A readme file answers questions about installing the versions of software needed for the project to run. The configuration steps are to be done so that everything runs smoothly. The readme file also answers questions basically based on which framework and coding language was used. It also helps the user understand why a certain framework was used instead of another. How the owner of the project got inspired, the method by which he solved the issue, and much more.
For our own development work:  A readme file provides two good use cases. First, if it is written prior to starting the development, it helps the developer by acting as a roadmap or tracker so that he is aware of what is left to do in the project life cycle. Second, it helps the developer understand the code that he might have written a few years back better when there is a need for it to be revisited and de-bugged. This helps a lot since it takes a lot of time to understand the code when you have to scrape through millions of lines of code that you might have written a few years back. The readme file helps you locate where each section of the code is and helps you find the piece of code you were looking for. It is a great time saver when the issue is critical and time-bound.
For other developers: The readme file serves a similar purpose to other developers going through the code and helps them understand where each section of code is located and which page contains the logic. This helps when there is an issue in the code, and another developer is trying to solve the issue. He can go through the code easily and find where the issue is. Likewise, if another developer is trying to enhance the code, then a good readme file goes a long way in helping that person.
So to wrap it all up. First, we have gone through the definition of the readme. The format in which it is written. Common syntax to be used, which will help you in writing one. Things to keep in mind while writing a readme file. A common template to follow and understand what to write and what not to write in a readme file. And last, we saw how the readme file is useful for different types of readers/users. 

So we hope this has helped you get a little bit of knowledge on the Readme file. To learn more about programming and other related concepts, check out the courses on Great Learning Academy. 

Have a Great Learning!